In [1]:
!pip install datasets evaluate transformers[sentencepiece] seqeval[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=466fec09b1741b05ac68bf3ef4bc45399391f0ae08d3eb7536309f0532bab640
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
from transformers import BertTokenizerFast, BertForTokenClassification, BertForSequenceClassification
import torch
import numpy as np
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Dataset
from transformers import TextDataset

In [3]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 1.4 MB/s eta 0:00:00


In [4]:
import json
# Read the JSON file
path = "/content/drive/MyDrive/model/B_data.json"
with open(path, 'r') as f:
    dataset = json.load(f)

In [5]:
path = "/content/drive/MyDrive/model/training_set.json"
with open(path, 'r') as f:
    test_dataset = json.load(f)

In [6]:
text, intent, ner = [], [], []
for i in dataset:
    text.append(i['text'])
    intent.append(i['intent'])
    ner.append(i['entities'].split())

In [7]:
test_text, test_intent, test_ner = [], [], []
for i in test_dataset:
    test_text.append(i['text'])
    test_intent.append(i['intent'])
    test_ner.append(i['entities'].split())

In [8]:
unique_intents = set(intent)
num_intent_labels = len(unique_intents)

unique_intents, num_intent_labels

({"'Schedule Appointment'",
  "'Schedule Meeting'",
  "'Set Alarm'",
  "'Set Reminder'",
  "'Set Timer'"},
 5)

In [9]:
one_dimensional_ner = [tag for subset in ner for tag in subset ]
unique_ner = set(one_dimensional_ner)
num_ner_labels = len(unique_ner)
unique_ner, num_ner_labels

({'B-DATE',
  'B-DUR',
  'B-TASK',
  'B-TIME',
  'I-DATE',
  'I-DUR',
  'I-TASK',
  'I-TIME',
  'O'},
 9)

In [10]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
ner_model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=num_ner_labels)
intent_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_intent_labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
labels_to_ids_ner = {
    'O': 0,
    'B-DATE': 1,
    'I-DATE': 2,
    'B-TIME': 3,
    'I-TIME': 4,
    'B-TASK': 5,
    'I-TASK': 6,
    'B-DUR': 7,
    'I-DUR': 8
    }

ids_to_labels_ner = {v: k for k, v in labels_to_ids_ner.items()}
ids_to_labels_ner

{0: 'O',
 1: 'B-DATE',
 2: 'I-DATE',
 3: 'B-TIME',
 4: 'I-TIME',
 5: 'B-TASK',
 6: 'I-TASK',
 7: 'B-DUR',
 8: 'I-DUR'}

In [12]:
labels_to_ids_intent = {
    "'Schedule Appointment'": 0,
    "'Schedule Meeting'": 1,
    "'Set Alarm'": 2,
    "'Set Reminder'": 3,
    "'Set Timer'": 4
}

ids_to_labels_intent = {v: k for k, v in labels_to_ids_intent.items()}
ids_to_labels_intent

{0: "'Schedule Appointment'",
 1: "'Schedule Meeting'",
 2: "'Set Alarm'",
 3: "'Set Reminder'",
 4: "'Set Timer'"}

In [13]:
class dataset(Dataset):
    def __init__(self, text, intent, ner, tokenizer, max_len=128):
        self.len = len(text)
        self.text = text
        self.intent = intent
        self.ner = ner
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence, ner label, and intent_label
        sentence = self.text[index].strip()
        intent_label = self.intent[index].strip()
        ner_labels = self.ner[index]

        # step 2: use tokenizer to encode a sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" which highlights where each token starts and ends
        encoding = self.tokenizer(
            sentence,
            return_offsets_mapping=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_len
        )

        # step 3: create ner token labels only for first word pieces of each tokenized word
        tokenized_ner_labels = [labels_to_ids_ner[label] for label in ner_labels]
        # create an empty array of -100 of length max_length
        encoded_ner_labels = np.ones(len(encoding['offset_mapping']), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        prev = -1
        for idx, mapping in enumerate(encoding['offset_mapping']):
            if mapping[0] == mapping[1] == 0:
                continue
            if mapping[0] != prev:
                # overwrite label
                encoded_ner_labels[idx] = tokenized_ner_labels[i]
                prev = mapping[1]
                i += 1
            else:
                prev = mapping[1]

        # create intent token labels
        tokenized_intent_label = labels_to_ids_intent[intent_label]

        # step 4: turn everything into Pytorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['ner_labels'] = torch.as_tensor(encoded_ner_labels)
        item['intent_labels'] = torch.as_tensor(tokenized_intent_label)

        return item

    def __len__(self):
        return self.len

In [14]:
training_set = dataset(text, intent, ner, tokenizer)
test_set = dataset(test_text, test_intent, test_ner, tokenizer)

In [15]:
training_set[5]

{'input_ids': tensor([ 101, 2275, 2019, 8598, 2005, 1021, 2572, 1012,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0

Let us verify that the input ids and corresponding targets are correct:

In [16]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[20]['input_ids']), training_set[20]['ner_labels']):
    print(f"{token} -- {label}")

[CLS] -- -100
schedule -- 0
a -- 0
dentist -- 5
appointment -- 6
for -- 0
april -- 1
5th -- 2
at -- 0
11 -- 3
: -- -100
00 -- -100
in -- 4
the -- 4
morning -- 4
. -- -100
[SEP] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PA

In [17]:
# The dataset is small, batch_size of 1 would not impact the training time significantly
training_loader = DataLoader(training_set, batch_size=1)
test_loader = DataLoader(test_set, batch_size=1)

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [19]:
device

'cpu'

In [20]:
ner_model.to(device)
intent_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

The initial loss of the model should be close to -ln(1/num_labels)=-ln(1/9). In this case it is 2.20.
Why? Because we are using cross entropy loss. The cross entropy loss is defined as -ln(probability score of the model for the correct class). In the beginning, the weights are random, so the probability distribution for all of the classes for a given token will be uniform, meaning that the probability for the correct class will be near 1/9. The loss for a given token will thus be -ln(1/9). As PyTorch's CrossEntropyLoss (which is used by BertForTokenClassification) uses mean reduction by default, it will compute the mean loss for each of the tokens in the sequence for which a label is provided.

In [21]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["ner_labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = ner_model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(2.2084, grad_fn=<NllLossBackward0>)

The shape of logits must be __[batch_size, sequence_length, num_labels]__

In [22]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 9])

# Training

# Multi Task Learning Architecture

## Soft Parameter Sharing

In [23]:
class SoftParameterSharing(torch.nn.Module):
    def __init__(self, ner_model, intent_model, alpha):
        super(SoftParameterSharing, self).__init__()
        self.ner_model = ner_model
        self.intent_model = intent_model
        self.alpha = alpha

    def forward(self, input_ids, mask, ner_labels, intent_labels):
        ner_logits = self.ner_model(input_ids=input_ids, attention_mask=mask, labels=ner_labels)

        intent_logits = self.intent_model(input_ids=input_ids, attention_mask=mask, labels=intent_labels)

        return ner_logits, intent_logits

    def soft_penalty(self):
        a = 0
        penalty = 0.0
        for param_ner, param_intent in zip(self.ner_model.parameters(), self.intent_model.parameters()):
            if param_ner.shape != param_intent.shape:
                continue

            penalty += torch.norm(param_ner - param_intent, p=2) ** 2

        return self.alpha * penalty


In [24]:
model = SoftParameterSharing(ner_model, intent_model, 0.05)

In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [26]:
model

SoftParameterSharing(
  (ner_model): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_featur

In [27]:
def train(epoch):
    tr_ner_loss, tr_ner_accuracy = 0, 0
    tr_intent_loss, tr_intent_accuracy = 0, 0
    nb_tr_steps = 0
    tr_ner_preds, tr_ner_labels = [], []
    tr_intent_labels, tr_intent_predictions = [], []
    model.train()


    for idx, batch in enumerate(training_loader):
        ids = batch['input_ids'].to(device, dtype=torch.long)
        mask = batch['attention_mask'].to(device, dtype=torch.long)
        ner_labels = batch['ner_labels'].to(device, dtype=torch.long)
        intent_labels = batch['intent_labels'].to(device, dtype=torch.long)

        ner_logits, intent_logits = model(ids, mask, ner_labels, intent_labels)

        ner_loss = ner_logits.loss
        intent_loss = intent_logits.loss

        soft_penalty = model.soft_penalty()

        comb_loss = ner_loss + intent_loss + soft_penalty

        tr_ner_loss += ner_logits['loss']
        tr_intent_loss += intent_logits['loss']
        nb_tr_steps += 1

        if idx % 5 == 0:
            loss_ner_step = tr_ner_loss / nb_tr_steps
            loss_intent_step = tr_intent_loss / nb_tr_steps
            print(f"Training NER loss per {idx} training steps: {loss_ner_step}")
            print(f"Training INTENT loss per {idx} training steps: {loss_intent_step}")

        # compute training accuracy (FOR NER)
        flattened_ner_targets = ner_labels.view(-1) # shape (batch_size * seq_len)
        active_ner_logits = ner_logits.logits.view(-1, num_ner_labels) # shape (batch_size*seq_len, num_labels)
        flattened_ner_predictions = torch.argmax(active_ner_logits, axis=1) # shape (batch_size * seq_len)

        # compute accuracy only at active labels
        active_ner_accuracy = ner_labels.view(-1) != -100 # shape (batch_size, seq_len)
        ac_ner_labels = torch.masked_select(flattened_ner_targets, active_ner_accuracy)
        ner_predictions = torch.masked_select(flattened_ner_predictions, active_ner_accuracy)

        tr_ner_labels.extend(ac_ner_labels)
        tr_ner_preds.extend(ner_predictions)

        # compute accuracy for intent_model
        # I CAN MAKE THE CALCULATION MUCH EASIER
        # FIGURE IT OUT
        flattened_intent_targets = intent_labels.view(-1)
        active_intent_logits = intent_logits.logits.view(-1, num_intent_labels)
        flattened_intent_predictions = torch.argmax(active_intent_logits, axis=1)

        sample_intent_accuracy = intent_labels.view(-1)
        active_intent_accuracy = torch.ones_like(sample_intent_accuracy, dtype=torch.bool)

        ac_intent_labels = torch.masked_select(flattened_intent_targets, active_intent_accuracy)
        intent_predictions = torch.masked_select(flattened_intent_predictions, active_intent_accuracy)

        tr_intent_labels.extend(ac_intent_labels)
        tr_intent_predictions.extend(intent_predictions)

        tmp_tr_ner_accuracy = accuracy_score(ac_ner_labels.cpu().numpy(), ner_predictions.cpu().numpy())
        tmp_tr_intent_accuracy = accuracy_score(ac_intent_labels.cpu().numpy(), intent_predictions.cpu().numpy())


        tr_ner_accuracy += tmp_tr_ner_accuracy
        tr_intent_accuracy += tmp_tr_intent_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters = model.parameters(), max_norm=10
        )

        # backward pass
        optimizer.zero_grad()
        comb_loss.backward()
        optimizer.step()

    epoch_loss = (tr_ner_loss + tr_intent_loss) / nb_tr_steps
    tr_ner_accuracy = tr_ner_accuracy / nb_tr_steps
    tr_intent_accuracy = tr_intent_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training NER accuracy epoch: {tr_ner_accuracy}")
    print(f"Training INTENT accuracy epoch: {tr_intent_accuracy}")


In [28]:
for epoch in range(3):
    print(f"Training epoch: {epoch+1}")
    print("----------------------------")
    train(epoch)

Training epoch: 1
----------------------------
Training NER loss per 0 training steps: 2.2553861141204834
Training INTENT loss per 0 training steps: 1.1455878019332886
Training NER loss per 5 training steps: 2.056060552597046
Training INTENT loss per 5 training steps: 1.7735024690628052
Training NER loss per 10 training steps: 1.8900456428527832
Training INTENT loss per 10 training steps: 1.6942753791809082
Training NER loss per 15 training steps: 1.773932695388794
Training INTENT loss per 15 training steps: 1.6004977226257324
Training NER loss per 20 training steps: 1.711548924446106
Training INTENT loss per 20 training steps: 1.6128660440444946
Training NER loss per 25 training steps: 1.6755248308181763
Training INTENT loss per 25 training steps: 1.558272123336792
Training NER loss per 30 training steps: 1.635511875152588
Training INTENT loss per 30 training steps: 1.5511865615844727
Training loss epoch: 3.111560821533203
Training NER accuracy epoch: 0.5111357689929119
Training INTEN

# Evaluating the model

In [29]:
def eval(ner_model, intent_model, test_loader):
    model.eval()

    eval_ner_loss, eval_ner_accuracy = 0, 0
    eval_intent_loss, eval_intent_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_ner_preds, eval_ner_labels = [], []
    eval_intent_preds, eval_intent_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(test_loader):
            ids = batch['input_ids'].to(device, dtype=torch.long)
            mask = batch['attention_mask'].to(device, dtype=torch.long)
            ner_labels = batch['ner_labels'].to(device, dtype=torch.long)
            intent_labels = batch['intent_labels'].to(device, dtype=torch.long)

            ner_logits, intent_logits = model(input_ids=ids, mask=mask, ner_labels=ner_labels, intent_labels=intent_labels)

            eval_ner_loss += ner_logits['loss']
            eval_intent_loss += intent_logits['loss']
            nb_eval_steps += 1

            if idx % 5 == 0:
                loss_ner_step = eval_ner_loss / nb_eval_steps
                loss_intent_step = eval_intent_loss / nb_eval_steps
                print(f"Validation NER loss per {idx} training steps: {loss_ner_step}")
                print(f"Validation INTENT loss per {idx} training steps: {loss_intent_step}")

            # compute training accuracy (FOR NER)
            flattened_ner_targets = ner_labels.view(-1) # shape (batch_size * seq_len)
            active_ner_logits = ner_logits.logits.view(-1, num_ner_labels) # shape (batch_size*seq_len, num_labels)
            flattened_ner_predictions = torch.argmax(active_ner_logits, axis=1) # shape (batch_size * seq_len)

            # compute accuracy only at active labels
            active_ner_accuracy = ner_labels.view(-1) != -100 # shape (batch_size, seq_len)
            ac_ner_labels = torch.masked_select(flattened_ner_targets, active_ner_accuracy)
            ner_predictions = torch.masked_select(flattened_ner_predictions, active_ner_accuracy)

            eval_ner_labels.extend(ac_ner_labels)
            eval_ner_preds.extend(ner_predictions)

            # compute accuracy for intent_model
            # It is possible to calculate the accuracy easier
            flattened_intent_targets = intent_labels.view(-1)
            active_intent_logits = intent_logits.logits.view(-1, num_intent_labels)
            flattened_intent_predictions = torch.argmax(active_intent_logits, axis=1)

            sample_intent_accuracy = intent_labels.view(-1)
            active_intent_accuracy = torch.ones_like(sample_intent_accuracy, dtype=torch.bool)

            ac_intent_labels = torch.masked_select(flattened_intent_targets, active_intent_accuracy)
            intent_predictions = torch.masked_select(flattened_intent_predictions, active_intent_accuracy)

            eval_intent_labels.extend(ac_intent_labels)
            eval_intent_preds.extend(intent_predictions)

            tmp_eval_ner_accuracy = accuracy_score(ac_ner_labels.cpu().numpy(), ner_predictions.cpu().numpy())
            tmp_eval_intent_accuracy = accuracy_score(ac_intent_labels.cpu().numpy(), intent_predictions.cpu().numpy())

            eval_ner_accuracy += tmp_eval_ner_accuracy
            eval_intent_accuracy += tmp_eval_intent_accuracy

    v_ner_labels = [ids_to_labels_ner[id.item()] for id in eval_ner_labels]
    v_ner_predictions = [ids_to_labels_ner[id.item()] for id in eval_ner_preds]

    v_intent_labels = [ids_to_labels_intent[id.item()] for id in eval_intent_labels]
    v_intent_predictions = [ids_to_labels_intent[id.item()] for id in eval_intent_preds]

    v_ner_loss = eval_ner_loss / nb_eval_steps
    v_intent_loss = eval_intent_loss / nb_eval_steps
    eval_ner_accuracy = eval_ner_accuracy / nb_eval_steps
    eval_intent_accuracy = eval_intent_accuracy / nb_eval_steps
    print(f"Validation NER loss: {v_ner_loss}")
    print(f"Validation INTENT loss: {v_intent_loss}")
    print(f"Validation NER accuracy: {eval_ner_accuracy}")
    print(f"Validation INTENT accuracy: {eval_intent_accuracy}")

    return v_ner_labels, v_ner_predictions, v_intent_labels, v_intent_predictions

Test the results of the model's training on test dataset

In [30]:
v_ner_labels, v_ner_predictions, v_intent_labels, v_intent_predictions = eval(ner_model, intent_model, test_loader)

Validation NER loss per 0 training steps: 0.615532636642456
Validation INTENT loss per 0 training steps: 1.0722367763519287
Validation NER loss per 5 training steps: 0.8261163830757141
Validation INTENT loss per 5 training steps: 1.0298174619674683
Validation NER loss per 10 training steps: 0.7968384027481079
Validation INTENT loss per 10 training steps: 1.0658154487609863
Validation NER loss per 15 training steps: 0.7390947937965393
Validation INTENT loss per 15 training steps: 0.9361368417739868
Validation NER loss per 20 training steps: 0.7255468368530273
Validation INTENT loss per 20 training steps: 0.9012988209724426
Validation NER loss: 0.7448558211326599
Validation INTENT loss: 0.9032648205757141
Validation NER accuracy: 0.8102625383875385
Validation INTENT accuracy: 0.7083333333333334
